In [104]:
from random import uniform
import oqupy
import numpy as np
import inspect
from scipy.linalg import ishermitian,issymmetric

In [13]:
def ham(x,y,z):
    return x*oqupy.operators.sigma('x') \
            +y*oqupy.operators.sigma('y') \
            +z*oqupy.operators.sigma('z')

sys=oqupy.ParameterizedSystem(ham)
num_args = len(inspect.signature(ham).parameters)
num_steps = 1

In [30]:
paras=[[uniform(-5,5) for i in range(num_args)] for j in range(2*num_steps)]

In [135]:
# use finite differencing to construct the derivative of the first-step prepropagator
# in the list fd
parasarray=np.array(paras)
parashifts=np.identity(num_args)
gp=sys.get_propagators(0.1,parasarray)
firstpreprop=gp(0)[0]
h=1e-6
print(parasarray)
print()
fd=[] 
for i in range(num_args):
    shift=parashifts[i,:]
    parasarrayshifted=np.array(parasarray)
    parasarrayshifted[0,:]=parasarray[0,:]+h*shift
    gp2=sys.get_propagators(0.1,parasarrayshifted)
    shiftedfirstpreprop=gp2(0)[0]
    fd.append((shiftedfirstpreprop-firstpreprop)/h)

[[ 3.86429921  4.60444756  0.95377972]
 [ 4.58504039 -4.05893236 -2.22265819]]



In [127]:
# call system.py to get the derivatives of the first-step prepropagator
gpd=sys.get_propagator_derivatives(0.1,parasarray)
firstprepropderivs=gpd(0)[0]


In [132]:
# compare the two
print("Difference between fd and system.py (max abs)= ",np.max(np.abs(np.array(fd)-np.array(firstprepropderivs))))
print("Largest magnitude element of the derivative from finite differencing = ",np.max(np.abs(np.array(fd))))
print("Largest magnitude element of the derivative from system.py = ",np.max(np.abs(np.array(firstprepropderivs))))

Difference between fd and system.py (max abs)=  4.77782522612377e-07
Largest magnitude element of the derivative from finite differencing =  0.09408837807259753
Largest magnitude element of the derivative from system.py =  0.09408837777988445


In [136]:
# check equality to numerical tolerance. works for h=1e-6
assert(np.allclose(np.array(fd),np.array(firstprepropderivs)))

In [137]:
# the above would not test for transposition error if the results are symmetric
# however, they are not - check here 
temp=np.array(firstprepropderivs[0])
print("Maximum difference in opposing elements = ",np.max(np.abs(temp-temp.transpose())))
issymmetric(temp,rtol=1e-05, atol=1e-08)

Maximum difference in opposing elements =  0.04352260454648309


False